# RDD Lineage

Spark mantiene seguimiento del linaje de cada RDD: esto esm, la secuencia de transformaciones que resultan en cada RDD.
Cada operación en un RDD, re-computa el linaje entero por default (mientras que no se especifique la persistencia del RDD).


En el linaje de un RDD, cada RDD tendrá un RDD *padre* y/o un RDD *hijo*. Spark crea un DAG (directed acyclic graph/grafo acíclico dirigido) consistente de las dependencias entre RDDs. Los RDDs se procesan en escenas o fases (spark stages),  que son conjuntos de transformaciones. La dependencia entre RDDs y las fases puede ser estrecha (narrow dependencie) o amplia (wide dependencie).

---

La dependencia estrecha, se define por los siguientes acuerdos:
- Las operaciones se pueden colapsar en una sola escena. Por ejemplo, la operación en conjunto de `map()`  y `filter()` hacia elementos del mismo dataset se pueden procesar en el mismo pase de cada elemento del dataset,
- Solo un RDD hijo depende del RDD padre; en el caso de que un RDD es creado a partir de un archivo de texto (RDD padre) con un RDD hijo a ejecutar un conjunto de transformaciones en una sola fase.
- No se requiere revolver datos entre los nodos

Las operaciones estrechas se prefieren por que maximizan la ejecución en paralelo y minimizan el intercambio de datos entre nodos (shuffling), lo que puede convertirse en un cuello de botella y ser costoso.


---
La dependencia amplia, en contraste tienen los siguientes acuerdos:

- Las operaciones definen una nueva fase de cómputo y es seguro que requieran revolver datos entre nodos
- RDDs con multiples dependencias; por ejemplo, `join()` requiere que un RDD sea dependiente de dos o más RDDs padres.

Las operaciones amplias son inevitables cuando se agrupan, reducen o unen datasets. Siempre se debe tener en cuenta el overhead que involucran estas operaciones.

El linaje se puede visualizar mediante la herramienta **DAG Visualization** en el detalle de los trabajos o fases de la interfaz web de Spark.

![reduceByKey() DAG](https://github.com/israelzuniga/dlatam-bigdata-workshop/blob/master/notebooks/img/reduceByKey_DAG.png?raw=true)


# Fault Tolerance

Spark registra el linaje de cada RDD, incluyendo el linaje de todos los RDDs padre (y a su vez los de sus padres...). Cualquier RDD con todas sus particiones puede ser reconstruido al estado que estaba cuando se presento un fallo de Spark (por ejemplo, la desconexión de un nodo. Por que los RDDs son distribuidos, pueden tolerar y recuperarse en la falla de cualquier nodo.

## Funciones no deterministas y tolerancia a fallos.

El uso de **funciones no deterministas** en cualquier app de Spark (funciones que producen resultados distintos cuando reciben la misma entrada, por ejemplo, `random`) afectarán la habilidad de reconstruir RDDs a un estado consistente y repetible. Las complicaciones son mayores si una función no determinista es usada como condición que afecta la lógica o flujo del programa